<a href="https://colab.research.google.com/github/Katzenwerfer/Joey-ESRGAN_Colab/blob/main/Joey's_ESRGAN_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ESRGAN (Joey's fork) on Colab
The original repo is here: https://github.com/JoeyBallentine/ESRGAN <br> 
Notebook edited by Katzenwerfer (https://github.com/Katzenwerfer) <br>
Original notebook is here: https://colab.research.google.com/github/AhabbscienceStudioPak/ESRGAN/blob/master/ESRGAN_Colab.ipynb <br>
A cool notebook to interpolate video ;): https://colab.research.google.com/github/HeylonNHP/RIFE-Colab/blob/main/RIFE_Colab.ipynb

In [ ]:
#@title Use this if you want to know which GPU you were assigned { display-mode: "form" }

!nvidia-smi

In [ ]:
#@title Setup the repository, *check if the dependencies are installed*, etc. { display-mode: "form" }

!git clone https://github.com/JoeyBallentine/ESRGAN
!rm /content/ESRGAN/input/baboon.png
!rm /content/ESRGAN/output/baboon_ESRGAN.png

#_Dependency Checker
#_I think I'm to stupid to make a dependency checker, and most of them are already installed so it's probably something not necesary to have
Iffmpeg = False

In [ ]:
#@title Upload your file { display-mode: "form" }

!mkdir /content/input/
%cd /content/input/

from google.colab import files
uploaded = files.upload() 
for filename in uploaded.keys():
  print('User uploaded file "{name}" with {length} bytes'.format(name=filename, length=len(uploaded[filename])))

In [ ]:
#@title Pre-process video file (only if source is a video) { display-mode: "form" }

#_Work in progress...
#_Don't forget to add tile caching
if Iffmpeg == False:
  %cd /content/
  !echo "Downloading ffmpeg"
  !wget https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz
  !tar -xf ffmpeg-release-amd64-static.tar.xz
  !rm ffmpeg-release-amd64-static.tar.xz
  ffmpeg = "/content/ffmpeg-4.3.1-amd64-static/ffmpeg"
  ffprobe = "/content/ffmpeg-4.3.1-amd64-static/ffprobe"
  !echo "FFmpeg downloaded"
  Iffmpeg = True

!echo "Extracting frames"
!mkdir /content/video/
!mv /content/input/*.* /content/video/
#_Tried to add automatic framerate detection, but I'm too stupid to do it, + I don't know a lot about linux or python
#!ffprobe -v 0 -of csv=p=0 -select_streams v:0 -show_entries stream=r_frame_rate /content/video/*.* <framerate
#!echo "framerate"
!ffmpeg -i  /content/video/*.* "/content/input/%4d.png"
!echo "Done extracting the frames"

In [ ]:
#@title Upload your model { display-mode: "form" }

!rm -rf /content/ESRGAN/models/*.*
%cd /content/ESRGAN/models
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  print('User uploaded model "{name}" with {length} bytes'.format(name=filename, length=len(uploaded[filename])))

In [ ]:
#@title Upscale the file(s) { display-mode: "form" }

!cp /content/input/*.* /content/ESRGAN/input
!python /content/ESRGAN/upscale.py /content/ESRGAN/models/*.pth --input /content/ESRGAN/input/ --output /content/ESRGAN/output/
!mkdir /content/output
!cp /content/ESRGAN/output/*.* /content/output
!rm -rf /content/ESRGAN/input/*.*
!rm -rf /content/ESRGAN/output/*.*

In [ ]:
#@title Download your image { display-mode: "form" }

from google.colab import files
!zip -r output.zip /content/output
files.download('output.zip')

In [ ]:
#@title Process and download as video { display-mode: "form" }

#_Work in progress...
useNvenc = True #@param{type:"boolean"}
Audio = False #@param{type:"boolean"}
Format = "mp4" #@param ["mp4","mkv","webm"]
crf =  20#@param{type:"number"}
Vrate =  10#@param{type:"number"} 

#_Make video directory
!mkdir /content/output-video

#_Encode video
#_You know...something I hate is when things are too complex in a completely unnecesary way
#_like this, why can't I call a normal command inside a python if statement
#_it probably needs some sort of thing to call the shell, but all the solutions I found for that are quite over-complex
#_I will still add it, but for now enjoy your nvenc encoded video with no audio
#_THANKS PYTHON!!
#_ :( 
#if useNvenc == True:
#  if Audio == False:
#    import os
#    os.system('...
!ffmpeg -r $Vrate -i "/content/output/%4d.png" -c:v h264_nvenc -preset slow -cq $crf /content/output-video/output.$Format
#...')
#  elif Audio == True:
#    import os
#    os.system('!ffmpeg -thread_queue_size 1024 -r framerate -i /content/output/%4d.png -i /content/video/*.* -map 0:v -map 1:a -c:v h264_nvenc -preset slow -cq crf -c:a copy /content/output-video/output.mkv')
#elif useNvenc == False:
#  if Audio == False:
#    import os
#    os.system('!ffmpeg -r framerate -i "/content/output/%4d.png" -c:v libx264 -preset veyslow -crf crf /content/output-video/output.Format')
#  elif Audio == True:
#    import os
#    os.system('!ffmpeg -thread_queue_size 1024 -r framerate -i "/content/output/%4d.png" -i /content/video/*.* -map 0:v -map 1:a -c:v libx264 -preset veryslow -crf crf -c:a copy /content/output-video/output.mkv')

#_Download video
from google.colab import files
!zip -r output.zip /content/output-video
files.download('output.zip')

In [ ]:
#@title Check the video data, use this to identify the fps of your video { display-mode: "form" }
#@markdown To be honest, I hate that I need to add this, but it will take me some time to find out how to auto-detect the framerate
!ffprobe /content/video/*.*